# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_data_to_load = "../WeatherPy/weather_df_csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# google maps
gmaps.configure(api_key=g_key)

# weight and location
locations = dropna_weather_data[["Lat.", "Lon."]]
humidity = dropna_weather_data["Humidity"].astype(float)





# heatmap
fig = gmaps.figure()

# heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness"] == 0]
# max temp above 70 degrees F
vacation_above_70 = vacation_no_cloud[vacation_no_cloud["Max Temp."] > 70]
# max temp below 80 degrees F
vacation_below_80 = vacation_above_70[vacation_above_70["Max Temp."] < 80]
# wind speed below 10 mph
vacation_low_winds = vacation_below_80[vacation_below_80["Wind Speed"] < 10]
# humidity below 60 %
best_vacation = vacation_low_winds[vacation_low_winds["Humidity"] < 60]
# Set Index
indexed_best_vacation = best_vacation.reset_index()
del indexed_best_vacation["index"]
indexed_best_vacation

In [ ]:
vaca_locations = indexed_best_vacation[["Lat.", "Lon."]]

vaca_humidity = indexed_best_vacation["Humidity"].astype(float)

In [ ]:
# Heatmap
vaca_fig = gmaps.figure()

# Heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, dissipating=False, max_intensity=50, point_radius=2.5)
vaca_fig.add_layer(vaca_heat_layer)
vaca_fig

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
Hotels = []

for city in range(len(indexed_best_vacation["City"])):

    lat = indexed_best_vacation.loc[city]["Lat."]
    lng = indexed_best_vacation.loc[city]["Lon."]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("nearest hotel not found")

# Dataframe with nearest hotel
indexed_best_vacation["Nearest Hotel"] = hotels
indexed_best_vacation

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
